# RAG using local models



https://python.langchain.com/docs/use_cases/question_answering/local_retrieval_qa

In [1]:
!pip install bs4 langchain fake_useragent elasticsearch eland[pytorch]==8.10.1 --user


In [22]:
# from langchain.document_loaders import WebBaseLoader

# loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
# loader.requests_kwargs = {'verify':False}

# data = loader.load()

# from langchain.text_splitter import CharacterTextSplitter

# text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# all_splits = text_splitter.split_documents(data)

c:\Users\refar\.conda\envs\es\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'lilianweng.github.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Created a chunk of size 2731, which is longer than the specified 500
Created a chunk of size 903, which is longer than the specified 500
Created a chunk of size 706, which is longer than the specified 500
Created a chunk of size 1538, which is longer than the specified 500
Created a chunk of size 903, which is longer than the specified 500
Created a chunk of size 1380, which is longer than the specified 500
Created a chunk of size 623, which is longer than the specified 500
Created a chunk of size 542, which is longer than the specified 500
Created a chunk of size 2352, which is longer than the specified 500
Created a chunk of size 1953, which is longer than the speci

In [3]:
from getpass import getpass
from langchain.vectorstores import ElasticsearchStore


In [6]:
# ES_URL = input('Elasticsearch URL: ')
ES_URL = input('Elasticsearch URL(ex:https://127.0.0.1:9200)')
ES_USER = "elastic" 
ES_USER_PASSWORD = getpass('elastic user PW: ')
CERT_PATH = input('Elasticsearch pem 파일 경로: ')
# pem 생성 방법: https://cdax.ch/2022/02/20/elasticsearch-python-workshop-1-the-basics/

In [14]:
from elasticsearch import Elasticsearch

client = Elasticsearch(
    ES_URL,
    basic_auth=(ES_USER, ES_USER_PASSWORD),
    ca_certs=CERT_PATH
)

In [ ]:
# !eland_import_hub_model --url "https://127.0.0.1:9200" -u elastic -p elastic --ca-certs "D:\es\8.10.2\kibana-8.10.2\data\ca_1695716998259.crt" --hub-model-id "d:/Projects/es-lab-kr/s-core/models/distilbert-base-multilingual-cased" --task-type text_embedding --start --clear-previous

In [8]:
import os
cwd = os.getcwd()

try :
    os.mkdir(cwd + "/models") 
except:
    pass

In [9]:
os.chdir(cwd + "/models")

try :
    os.system("git clone https://huggingface.co/distilbert-base-multilingual-cased")
except:
    print('이미 모델이 존재합니다.')

os.chdir(cwd)

In [10]:
import os
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.common import es_version
from eland.ml.pytorch.transformers import TransformerModel

# 현재 경로 얻기
cwd = os.getcwd()
local_model_path = cwd + '/models/distilbert-base-multilingual-cased'


print(local_model_path)

# 모델 이름 및 작업 유형 설정
#tm = TransformerModel(local_model_path, "text_embedding")
es_cluster_version = es_version(client)
tm = TransformerModel(model_id=local_model_path, task_type="text_embedding", es_version=es_cluster_version)

# Elasticsearch용 modelID를 설정
es_model_id = "sentence-transformers_distilbert-base-multilingual-cased"

tmp_path = "models/sentence-transformers_distilbert-base-multilingual-cased"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

ptm = PyTorchModel(client, es_model_id)
ptm.stop()
ptm.delete()

ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config) 

ptm.start()

c:\Users\refar\.conda\envs\es\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


c:\Users\refar\Projects\es-lab-kr\s-core/models/distilbert-base-multilingual-cased


No sentence-transformers model found with name c:\Users\refar\Projects\es-lab-kr\s-core/models/distilbert-base-multilingual-cased. Creating a new one with MEAN pooling.
Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
c:\Users\refar\.conda\envs\es\lib\site-packages\transformers\models\distilbert\modeling_distilbert.py:223: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)
100%|██████████| 515/515 [01:04<00:00,  8.02 parts/s]


In [66]:
from langchain.vectorstores import ElasticsearchStore
from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

embeddings = ElasticsearchEmbeddings.from_es_connection(
    es_connection=client,
    model_id = es_model_id
)

vectorstore = ElasticsearchStore(
    es_connection=client, 
    embedding=embeddings, 
    index_name= "workplace_index"
)


In [67]:
from urllib.request import urlopen
from langchain.text_splitter import CharacterTextSplitter
import json

url = "./data.json"

response = urlopen("https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/example-apps/workplace-search/data/data.json")

workplace_docs = json.loads(response.read())

workplace_docs

[{'content': "Effective: March 2020\nPurpose\n\nThe purpose of this full-time work-from-home policy is to provide guidelines and support for employees to conduct their work remotely, ensuring the continuity and productivity of business operations during the COVID-19 pandemic and beyond.\nScope\n\nThis policy applies to all employees who are eligible for remote work as determined by their role and responsibilities. It is designed to allow employees to work from home full time while maintaining the same level of performance and collaboration as they would in the office.\nEligibility\n\nEmployees who can perform their work duties remotely and have received approval from their direct supervisor and the HR department are eligible for this work-from-home arrangement.\nEquipment and Resources\n\nThe necessary equipment and resources will be provided to employees for remote work, including a company-issued laptop, software licenses, and access to secure communication tools. Employees are respo

In [63]:
metadata = []
content = []

for doc in workplace_docs:
  content.append(doc["content"])
  metadata.append({
      "name": doc["name"],
      "summary": doc["summary"],
      "rolePermissions":doc["rolePermissions"],
  })

text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=0)
docs = text_splitter.create_documents(content, metadatas=metadata)

Created a chunk of size 866, which is longer than the specified 800
Created a chunk of size 1120, which is longer than the specified 800


In [69]:
db = vectorstore.from_documents(
    docs, 
    embeddings, 
    es_connection=client,
    index_name= "workplace_index"
)

ConnectionTimeout: Connection timed out

In [72]:
vectorstore.client.indices.refresh(index="workplace_index")

query = "What did the president say about Ketanji Brown Jackson"
results = vectorstore.similarity_search(query)
print(results)

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [workplace_index]', workplace_index, index_or_alias)